# Function Calling with Gemini — Run in Google Colab

➡ **Open this notebook in Google Colab.**

### Watch the series on YouTube
- Part 1: https://youtu.be/sD6czjVekxk
- Part 2: https://youtu.be/IWpq4KubUCc
- Part 1: https://youtu.be/ePQ_vFhHZfs

**Setup tip:** In Colab, add a secret `GEMINI_API_KEY`

# Function Calling
**Function calling** (also known as tool calling) provides a powerful and flexible way to interface with external systems and access data outside their training data. Function calling has 3 primary use cases:
<br>

* **Augment Knowledge**: Access information from external sources like databases, APIs, and knowledge bases.
* **Extend Capabilities**: Use external tools to perform computations and extend the limitations of the model, such as using a calculator or creating charts.
* **Take Actions**: Interact with external systems using APIs, such as scheduling appointments, creating invoices, sending emails, or controlling smart home devices.

## Terminology

### Tools - functionality we give the model


*   What is it:
    * It’s a specific **capability** you tell the model it can use—think of it as a button the model is allowed to press.
    * As the model generates a response to your prompt, it may **decide it needs that capability** (data or an action) to follow your instructions accurately.

*   When does model use it:
    * Sometimes the prompt requires **fresh data** (today’s weather, an account balance) or a **real action** (issuing a refund).
    * The model recognizes that its own knowledge isn’t enough and **chooses a tool you’ve exposed** to complete the task.

* Examples of tools you might expose
  * Get today’s weather for a specific location.
  * Access account details for a given user ID.
  * Issue a refund for a lost order.

In practice, it can be anything you want the model to know or do while responding—APIs, databases, internal services, automations. <br><br>

### Tool calls - requests from the model to use tools

* What it is: **A special kind of model response** where, after reading your prompt, the model decides it needs one of your tools to follow the instructions.

* When it happens: During generation, the model inspects the prompt and concludes that **external data or an action is required** (something it can’t reliably produce from training data alone).

**Plain-English example**
```
Prompt: “What is the weather in Paris?”

Model’s response (conceptually): “I should call get_weather with { location: 'Paris' }.”
```
<br><br>
### Tool call outputs - output we generate for the model
* What the model returns: Only **structured data** with the **tool name** and **parameters** (e.g., get_current_weather(location="Boston")).

* What we must do: **Our application executes the tool**, produces an **output** (JSON or text), and then **feeds that output back to the model** so it can finish the reply.

* Why this matters: This bridges the model to our **systems/APIs**—the model doesn’t call external services by itself.

**Weather example**

Prompt: ```“What’s the weather in Paris?”```


* Model’s tool call (proposal): ```get_weather({ "location": "Paris" })``` (model does not execute it)

* We execute: Call our ```get_weather``` service and get:
```
{ "temperature": "25", "unit": "C" }
```


* We return to the model: tool definition + original prompt + model’s tool call + our tool output.

* Model’s final answer: ```“The weather in Paris today is 25 C.”```
<br><br>

### Functions vs. tools — what’s the difference?

* **Tool (umbrella term)**:
Any capability you tell the model it can use. The model may request a tool; your app (or platform) actually runs it.

  * **Function** (a schema-defined tool):
  A tool described with a JSON schema (name + typed parameters). The model returns function name + args; your code executes and sends the result back. Great for predictable, validated inputs.

  * **Custom tools (free-text I/O)**:
  Tools that take and/or return plain text instead of strict JSON (Python code as a response). Useful when the interface is flexible or hard to formalize. You still run them on your side.

  * **Built-in platform tools**:
  Tools provided by the platform (e.g., web search, code execution, MCP server access). You enable/configure them; the model can call them without you writing the tool backend.


* **Execution model (always)**:
The model doesn’t execute tools. It proposes a call; you (or the platform) run it, then you return the output so the model can finish the answer.



## Function Calling Flow
![Diagram](../images/function-calling.png)

1. **User → Application: Prompt arrives**

* The user asks something.

* Example: ```“What’s the weather in Paris right now?” ```
<br>

2. **Application → Model: Send prompt + tool list**

* Your app forwards the prompt and the available tools (function declarations).

* Example tools: ```get_weather(location), convert_currency(amount, from, to)```
<br>

3. **Model → Application: Proposes a tool call**

* The model does not run code. It returns name + JSON args for a tool if needed.

* Example function call: ``` { "name": "get_weather", "args": { "location": "Paris, FR" } } ```
<br>

4. **Application → API: You execute the tool**

* Your code validates args and calls your API/service.

* Example request: ```GET /weather?location=Paris%2C%20FR```

* Example API response: ``` { "temperature": 25, "unit": "C", "condition": "Partly cloudy" }```
<br>

5. **Application → Model: Return the tool result**

* Send back the function response (and the original tool call) so the model can use real data.

* Example function response payload: ```{ "get_weather": { "temperature": 25, "unit": "C", "condition": "Partly cloudy" } }```
<br>

6. **Model → Application: Finalize the answer**

* The model writes a natural-language reply (or may request another tool if needed).

* Example final text: ```“It’s 25 °C, partly cloudy in Paris.”```
<br>

7. **Application → User: Deliver the response**

* Your app returns the final answer (and optionally, structured data).

* Example user message: ```“Paris weather: 25 °C, partly cloudy.”```

<br><br>


## Setup (Gemini API key)
We’ll use the Gemini API via the ```google.genai``` Python package. Grab a free API key from [Google AI Studio](https://aistudio.google.com/)

## Example - Function calling flow

### 1) Define function declarations (OpenAPI-style)

* **What you define**: a function tool (set_light_values) and its JSON schema parameters.

* **What the model sees**: name, description, and typed args it can propose



In [ ]:
# Define a function that the model can call to control smart lights
set_light_values_declaration = {
    "name": "set_light_values",
    "description": "Sets the brightness and color temperature of a light.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "integer",
                "description": "Light level from 0 to 100. Zero is off and 100 is full brightness",
            },
            "color_temp": {
                "type": "string",
                "enum": ["daylight", "cool", "warm"],
                "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.",
            },
        },
        "required": ["brightness", "color_temp"],
    },
}

# This is the actual function that would be called based on the model's suggestion
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {"brightness": brightness, "colorTemperature": color_temp}


### 2) Call the model with tool declarations

**What happens**: you send prompt + tools; the model may return a function call suggestion (name + args).

**Your next step**: inspect the function call proposal.

In [ ]:
from google import genai
from google.genai import types
from google.colab import userdata

# Configure the client and tools
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))

# ###### Uncomment to Auto-generate a FunctionDeclaration from your callable (function) ######
# fn_decl = types.FunctionDeclaration.from_callable(callable=set_light_values, client=client)
# tool = types.Tool(function_declarations=[fn_decl])
# ##############

tool = types.Tool(function_declarations=[set_light_values_declaration])
config = types.GenerateContentConfig(tools=[tool])

# Define user prompt
contents = [
    types.Content(
        role="user", parts=[types.Part(text="Turn the lights down to a romantic level")]
    )
]

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config,
)

# Response schema: https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentResponse

print(response.candidates[0].content.parts[0].function_call)
# response

id=None args={'brightness': 20, 'color_temp': 'warm'} name='set_light_values'


### 3) Execute your function (the model doesn’t)

* **What you do**: parse the name + args, run your real function, capture the result.

In [ ]:
# Extract tool call details, it may not be in the first part.
tool_call = response.candidates[0].content.parts[0].function_call

if tool_call.name == "set_light_values":
    result = set_light_values(**tool_call.args)
    print(f"Function execution result: {result}")


Function execution result: {'brightness': 20, 'colorTemperature': 'warm'}


### 4) Return the function result to the model to finalize

* **Why**: the model uses the tool output to compose the final natural-language response.

* **What you send**: original model content + a function response part.

In [ ]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=tool_call.name,
    response={"result": result},
)

# Append function call and result of the function execution to contents
contents.append(response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

final_response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=config,
    contents=contents,
)

print(final_response.text)


Alright, I've set the lights to a romantic level for you!


## Function declaration in the OpenAPI schema format

* **Purpose**: Tell Gemini **what tools exist** and **how to call them**.

* **Where**: You pass a ```Tool``` that contains one or more function declarations.

* **Format**: Functions are defined in **JSON** using a **subset of OpenAPI/JSON Schema** for parameters.

**Fields you must define**

1) **name** (string) — unique, descriptive

    * Use letters/nums/underscores/camelCase; no spaces.

    * Example: get_weather_forecast, sendEmail.

2) **description** (string) — when to use it

    * Explain the purpose and capabilities; be specific.

    * Example: “Finds theaters by location and optional movie title currently in theaters.”

3) **parameters** (object) — input schema

    * Describes the arguments the model may supply.

* Inside parameters:

  * **type** (string) — overall type, usually "object".

  * **properties** (object) — each parameter is a property with its own schema:

    * **type** — e.g., "string", "integer", "boolean", "array".

    * **description** — what it is, format rules, examples.

      * Example: “City and state, e.g., San Francisco, CA or ZIP like 95616.”

    * **enum** (optional) — fixed allowed values (improves accuracy).

        * Example: "enum": ["daylight","cool","warm"]

  * **required** (array) — which parameter names are mandatory.

 *Complete list of supported keywords of the OpenAPI schema format [here](https://ai.google.dev/api/caching#Schema)*



---


**Schema fields**

* type — data type (string, integer, number, boolean, array, object).

* title — short human title for the schema.

* description — plain-English help (Markdown ok).

* nullable — whether a value can be null.

* properties — fields of an object (each with its own schema).

* required — property names that must be present.

* minProperties / maxProperties — bounds on the number of properties in an object.

* minLength / maxLength / pattern — string length and regex constraints.

* enum — fixed set of allowed values (improves reliability).

* format — optional refinement hint (e.g., int32).

* items / minItems / maxItems — array element schema and size limits.

* minimum / maximum — numeric bounds for numbers/integers.

* anyOf — input is valid if it matches any one branch.

* default — documentation-only default; doesn’t enforce or auto-fill.

* example — example value for docs.

* propertyOrdering — display/order hint (non-standard).



---
<br>

**NOTE** : You can also **construct FunctionDeclarations from Python functions directly** using types.FunctionDeclaration.from_callable(client=client, callable=your_function)
<br>

**Manual VS Automated Function Declaration - Decision checklist**

  * Use from_callable if:

    * You need to ship a demo.

    * Your inputs are simple (few fields, no complex constraints).

    * You want auto-run with minimal plumbing.

  * Use JSON Schema if:

    * You need strict validation (enums, ranges, either/or).

    * You care about consistency across teams/languages.

    * You’re going production (quotas, safety, logging, stable interface).


In [ ]:
types.FunctionDeclaration.from_callable(callable=set_light_values, client=client)

FunctionDeclaration(
  description="""Set the brightness and color temperature of a room light. (mock API).

Args:
    brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
    color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

Returns:
    A dictionary containing the set brightness and color temperature.""",
  name='set_light_values',
  parameters=Schema(
    properties={
      'brightness': Schema(
        type=<Type.INTEGER: 'INTEGER'>
      ),
      'color_temp': Schema(
        type=<Type.STRING: 'STRING'>
      )
    },
    required=[
      'brightness',
      'color_temp',
    ],
    type=<Type.OBJECT: 'OBJECT'>
  )
)

### Example - OpenAPI schema format

* We’re giving the model a ```tool``` called get_weather so it can get current weather or a short forecast for a location (or coords).


* In this case, we provide the flexibility to specify the location **either by its name** or **coordinates**. It can also add optional days (how many days ahead) and tags (like filters).

* When a user asks, “What’s the weather in Paris?”, the model doesn’t hit an API itself. It proposes a call like get_weather({ "location": "Paris, FR", "days": 1 }).

In [ ]:
get_weather_declaration = {
  "name": "get_weather",  # function name the model will call
  "description": "Get current weather or a short forecast for a location (or coords).",
  "parameters": {
    "type": "object",
    "title": "Args for get_weather",
    "description": "Inputs for get_weather.",
    "nullable": False,

    "properties": {
      "location": {
        "type": "string",
        "description": "City/state or ZIP (e.g., 'San Francisco, CA' or '95616').",
        "minLength": 1,
        "maxLength": 100,
        "pattern": ".*",
        "enum": ["San Francisco, CA", "Paris, FR", "95616"],
        "nullable": False,
        "default": "San Francisco, CA",
        "example": "Paris, FR"
      },

      "days": {
        "type": "integer",
        "description": "Days ahead (1–7).",
        "format": "int32",
        "minimum": 1,
        "maximum": 7,
        "nullable": False
      },

      "tags": {
        "type": "array",
        "description": "Optional filters.",
        "items": { "type": "string" },
        "minItems": 0,
        "maxItems": 10,
        "nullable": True
      },

      "coords": {
        "type": "object",
        "description": "Geographic coordinates.",
        "properties": {
          "lat": { "type": "number", "minimum": -90,  "maximum": 90  },
          "lon": { "type": "number", "minimum": -180, "maximum": 180 }
        },
        "required": ["lat", "lon"]
      }
    },


    "minProperties": 1,
    "maxProperties": 10,

    "anyOf": [
      {
        "type": "object",
        "properties": { "location": { "type": "string" } },
        "required": ["location"]
      },
      {
        "type": "object",
        "properties": {
          "coords": {
            "type": "object",
            "properties": {
              "lat": { "type": "number", "minimum": -90,  "maximum": 90  },
              "lon": { "type": "number", "minimum": -180, "maximum": 180 }
            },
            "required": ["lat", "lon"]
          }
        },
        "required": ["coords"]
      }
    ],

    "propertyOrdering": ["location", "days", "tags", "coords"]
  }
}

In [ ]:
from google import genai
from google.genai import types
from google.colab import userdata

# Configure the client and tools
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))
tools = types.Tool(function_declarations=[get_weather_declaration])
config = types.GenerateContentConfig(tools=[tools])

### Scenarios ###
# - City input path
# Prompt: “Weather in Paris today.”
# Matches the branch that requires location.

# - Coordinates path
# Prompt: “Forecast near 48.8566, 2.3522 for 3 days.”
# Matches the branch that requires coords.

# - Validation failure
# Prompt: “Show forecast.” (no city/coords)
# Your app should either ask a clarifying question or let the model ask for the missing field, based on your prompting/settings.
#################

# Define user prompt
contents = [
    types.Content(
        role="user", parts=[types.Part(text="Show forecast for paris.")]
    )
]

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config,
)

print(response.candidates[0].content.parts[0].function_call)
# print(response.candidates[0].content.parts[0].text)

id=None args={'location': 'Paris, FR'} name='get_weather'


## Automatic function calling (Python only)
**What it is**

* Pass **Python functions directly** as tools; the SDK turns them into function declarations, **executes them** when the model asks, and returns the final text to you.

* Write functions with **type hints** and a **clear docstring** [Google-style recommended](https://google.github.io/styleguide/pyguide.html#383-functions-and-methods)
<br>

**What the SDK does for you**

* Detects when the model wants to call a function.

* Calls your Python function with the model’s args.

* Sends back your function’s result to the model.

* Returns the model’s final text response to your code.
<br>

**What types the SDK can describe automatically**
* The Python SDK can auto-build a tool schema from your **type hints**.
* It understands scalars, lists (even nested), and **Pydantic models**.
* It does **not** do a good job with raw dict[...]
* For strict rules (enums, ranges, either/or), write a JSON schema yourself.

<br>




In [ ]:
# ❌ The SDK can’t derive clear, per-field structure from a dict, so you’ll typically see a very loose/ambiguous schema
# The model won’t know which keys to supply, so calls will be vague or malformed.

def set_coords_bad(coords: dict[str, float]) -> str:
    """Stores coordinates. coords must include keys 'lat' and 'lon'."""
    return "ok"


bad_decl = types.FunctionDeclaration.from_callable(callable=set_coords_bad, client=client)
bad_decl

FunctionDeclaration(
  description="Stores coordinates. coords must include keys 'lat' and 'lon'.",
  name='set_coords_bad',
  parameters=Schema(
    properties={
      'coords': Schema(
        type=<Type.OBJECT: 'OBJECT'>
      )
    },
    required=[
      'coords',
    ],
    type=<Type.OBJECT: 'OBJECT'>
  )
)

In [ ]:
# ✅ Pydantic gives the SDK a named-object shape with required fields, so the model can reliably supply the right arguments.

from pydantic import BaseModel

class Coords(BaseModel):
    lat: float
    lon: float

def set_coords_good(coords: Coords) -> str:
    """Stores coordinates (lat/lon)."""
    return "ok"

good_decl = types.FunctionDeclaration.from_callable(callable=set_coords_good, client=client)
good_decl

FunctionDeclaration(
  description='Stores coordinates (lat/lon).',
  name='set_coords_good',
  parameters=Schema(
    properties={
      'coords': Schema(
        properties={
          'lat': Schema(
            type=<Type.NUMBER: 'NUMBER'>
          ),
          'lon': Schema(
            type=<Type.NUMBER: 'NUMBER'>
          )
        },
        required=[
          'lat',
          'lon',
        ],
        type=<Type.OBJECT: 'OBJECT'>
      )
    },
    required=[
      'coords',
    ],
    type=<Type.OBJECT: 'OBJECT'>
  )
)

### Example - Automatic Function Calling

In [ ]:
from google import genai
from google.genai import types

# Define the function with type hints and docstring
def get_current_temperature(location: str) -> dict:
    """Gets the current temperature for a given location.

    Args:
        location: The city and state, e.g. San Francisco, CA

    Returns:
        A dictionary containing the temperature and unit.
    """
    # ... (implementation) ...
    return {"temperature": 25, "unit": "Celsius"}

# Configure the client
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))
config = types.GenerateContentConfig(
    tools=[get_current_temperature]
)  # Pass the function itself

# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What's the temperature in Boston?",
    config=config,
)

print(response.text)  # The SDK handles the function call and returns the final text

The current temperature in Boston, MA is 25 degrees Celsius.


### **Variations you might see**

* **No-tool path**: If the prompt doesn’t need external data, the model replies directly with text.

* **Clarifying questio**n: If required args are missing (“What’s the weather?”), the model may ask: “Which city?”—because your schema requires location.


* **Parallel function calling**: The model can propose multiple tool calls in one turn. Great for independent work: fetching data from multiple sources or triggering separate actions at once (e.g., CRM + Billing lookups; weather for many cities). Used when the functions are not dependent on each other.


In [ ]:
# PARALLEL FUNCITON CALLING EXAMPLE using automated function calling
# Use Case: converting your apartment into a disco.

from google import genai
from google.genai import types

# Actual function implementations
def power_disco_ball_impl(power: bool) -> dict:
    """Powers the spinning disco ball.

    Args:
        power: Whether to turn the disco ball on or off.

    Returns:
        A status dictionary indicating the current state.
    """
    return {"status": f"Disco ball powered {'on' if power else 'off'}"}

def start_music_impl(energetic: bool, loud: bool) -> dict:
    """Play some music matching the specified parameters.

    Args:
        energetic: Whether the music is energetic or not.
        loud: Whether the music is loud or not.

    Returns:
        A dictionary containing the music settings.
    """
    music_type = "energetic" if energetic else "chill"
    volume = "loud" if loud else "quiet"
    return {"music_type": music_type, "volume": volume}

def dim_lights_impl(brightness: float) -> dict:
    """Dim the lights.

    Args:
        brightness: The brightness of the lights, 0.0 is off, 1.0 is full.

    Returns:
        A dictionary containing the new brightness setting.
    """
    return {"brightness": brightness}

# Configure the client
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))
config = types.GenerateContentConfig(
    tools=[power_disco_ball_impl, start_music_impl, dim_lights_impl]
)

# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Do everything you need to this place into party!",
    config=config,
)


print(response.text)
# print(response)

Alright, the disco ball is spinning, the music is loud and energetic, and the lights are dimmed. The party is ready to start!


* **Compositional function calling** :
  - Compositional (sequential) function calling allows Gemini to chain multiple function calls together to fulfill a complex request.
  - For example, to answer "Get the temperature in my current location", the Gemini API might first invoke a get_current_location() function followed by a get_weather() function that takes the location as a parameter.

## Function calling modes
The Gemini API lets you control how the model uses the provided tools (function declarations). Specifically, you can set the mode within the ```.function_calling_config```.

* **AUTO (Default)**: The model decides whether to generate a natural language response or suggest a function call based on the prompt and context. **This is the most flexible mode and recommended for most scenarios.**

* **ANY**: The model is constrained to always predict a function call and guarantees function schema adherence. If ```allowed_function_names``` is not specified, the model can choose from any of the provided function declarations. If ```allowed_function_names``` is provided as a list, the model can only choose from the functions in that list. **Use this mode when you require a function call response to every prompt (if applicable)**.

* **NONE**: The model is prohibited from making function calls. This is equivalent to sending a request without any function declarations. **Use this to temporarily disable function calling without removing your tool definitions**.

## Best practices
* **Function and Parameter Descriptions**: Be extremely clear and specific in your descriptions. The model relies on these to choose the correct function and provide appropriate arguments.
* **Naming**: Use descriptive, no spaces/dots/dashes.
Good: ```getWeatherForecast / set_light_values``` • Bad: ```Get Weather! or get-weather```
* **Strong typing**: Prefer specific types; use ```enum``` for closed sets.
e.g., ```units: {type:"string", enum:["C","F"]}```; ```days: {type:"integer", minimum:1, maximum:7}```
* **Tool selection** (keep it tight): Only include tools relevant to the turn (ideally ≤ 10–20 active)—omit unrelated tools.
* **Prompt engineering**
  * provide context:
e.g., System: “You are a helpful weather assistant.”

  * instruct usage: e.g., ”Use ```get_weather``` for real conditions. Don’t guess; if date missing, ask.”

  * encourage clarification: e.g., “If location is missing or ambiguous, ask the user to specify city or coords.”
* **Temperature** (determinism): Use low temp for tool calls.
e.g., ```temperature=0``` → fewer hallucinated args.
* **Validation** (for impactful actions): Confirm before executing.
e.g., “About to **refund $129** for order **#A123**—should I proceed? (yes/no)”
* **Error handling** (graceful, structured): Return actionable errors the model can summarize.
e.g., ```{"error":"inventory_timeout","hint":"Try warehouse='DAL' or retry in 30s"}```
* **Token Limits**: Function descriptions and parameters count towards your input token limit - trim if hitting token limits
